In [20]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("ecommerce.db")


In [22]:
orders = pd.read_csv("../data/raw/olist_orders_dataset.csv")
customers = pd.read_csv("../data/raw/olist_customers_dataset.csv")
payments = pd.read_csv("../data/raw/olist_order_payments_dataset.csv")
reviews = pd.read_csv("../data/raw/olist_order_reviews_dataset.csv")

orders.to_sql("orders", conn, if_exists="replace", index=False)
customers.to_sql("customers", conn, if_exists="replace", index=False)
payments.to_sql("payments", conn, if_exists="replace", index=False)
reviews.to_sql("reviews", conn, if_exists="replace", index=False)


99224

In [23]:
pd.read_sql_query("SELECT COUNT(*) FROM orders", conn)


,COUNT(*)
0,99441


In [24]:
query_orders_per_customer = """
SELECT
    c.customer_unique_id,
    COUNT(DISTINCT o.order_id) AS total_orders
FROM orders o
JOIN customers c
    ON o.customer_id = c.customer_id
GROUP BY c.customer_unique_id
"""
orders_per_customer = pd.read_sql_query(query_orders_per_customer, conn)
orders_per_customer.head()


,customer_unique_id,total_orders
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1


In [25]:
query_with_payments = """
SELECT
    c.customer_unique_id,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(p.payment_value) AS total_spent
FROM orders o
JOIN customers c
    ON o.customer_id = c.customer_id
LEFT JOIN payments p
    ON o.order_id = p.order_id
GROUP BY c.customer_unique_id
"""
customer_spend = pd.read_sql_query(query_with_payments, conn)
customer_spend.head()


,customer_unique_id,total_orders,total_spent
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19
2,0000f46a3911fa3c0805444483337064,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,1,196.89


In [28]:
query_customer_base = """
SELECT
    c.customer_unique_id,
    COUNT(DISTINCT o.order_id) AS total_orders,
    SUM(p.payment_value) AS total_spent,
    AVG(r.review_score) AS avg_review_score
FROM orders o
JOIN customers c
    ON o.customer_id = c.customer_id
LEFT JOIN payments p
    ON o.order_id = p.order_id
LEFT JOIN reviews r
    ON o.order_id = r.order_id
GROUP BY c.customer_unique_id
"""
customer_base = pd.read_sql_query(query_customer_base, conn)
customer_base.head()


,customer_unique_id,total_orders,total_spent,avg_review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,1,141.90,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,27.19,4.0
2,0000f46a3911fa3c0805444483337064,1,86.22,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,1,43.62,4.0
4,0004aac84e0df4da2b147fca70cf8255,1,196.89,5.0


In [30]:

customer_base.isna().sum()


customer_unique_id      0
total_orders            0
total_spent             1
avg_review_score      716
dtype: int64

In [31]:
customer_base.shape

(96096, 4)

This notebook produces the customer-level analytical base table used for downstream feature engineering